# 1: 位置 $x$ と時刻 $t$ における波の振幅 $a(x, t)$ の式

$$
a(x, t) = A \sin(kx - \omega t + \phi) \hspace{30pt} (1) 
$$

ここで $A$ は振幅、$k$ は波数、$\omega$ は振動数、$\phi$ は位相を表します。

以下のコードでは波数、振動数、位相を変化させたときの波の形をアニメーションで描画します。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib import cm
from IPython.display import HTML
import matplotlib

# アニメーションの埋め込み制限を増やす
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

# 時空間での波の伝播可視化
# Y軸を時間軸として使用

# ハードコードされたパラメータ
A = 1.0                    # 振幅
k = 1.0                    # 波数
omega = 2.0                # 角周波数
phi = 0.0                  # 初期位相

# 導出される値
wavelength = 2 * np.pi / k
period = 2 * np.pi / omega
phase_velocity = omega / k

print(f"Wave parameters:")
print(f"  Wave number: k = {k:.2f} rad/m")
print(f"  Angular frequency: ω = {omega:.2f} rad/s")
print(f"  Wavelength: λ = {wavelength:.2f} m")
print(f"  Period: T = {period:.2f} s")
print(f"  Phase velocity: c = {phase_velocity:.2f} m/s")

# 空間と時間のグリッド
x = np.linspace(-5, 10, 200)
t = np.linspace(0, 10, 200)
X, T = np.meshgrid(x, t)

# 波動場の計算
# a(x,t) = A * sin(kx - ωt + φ)
Z = A * np.sin(k*X - omega*T + phi)

# 監視する2点
x1, x2 = 0, 2
idx1 = np.argmin(np.abs(x - x1))
idx2 = np.argmin(np.abs(x - x2))

# アニメーション版
fig_anim = plt.figure(figsize=(16, 10))

# アニメーション用のプロット
ax_contour = fig_anim.add_subplot(1, 2, 1)
ax_time = fig_anim.add_subplot(1, 2, 2)

# 固定された時間範囲
t_start = 0
t_end = 10
dt = 0.05
n_frames = int(t_end / dt)

# 固定されたグリッド（アニメーション全体で共通）
t_fixed = np.linspace(t_start, t_end, 200)
x_fixed = np.linspace(-5, 10, 200)
X_fixed, T_fixed = np.meshgrid(x_fixed, t_fixed)
Z_fixed = A * np.sin(k*X_fixed - omega*T_fixed + phi)

def init_anim():
    ax_contour.clear()
    ax_time.clear()
    return []

def animate(frame):
    ax_contour.clear()
    ax_time.clear()
    
    current_time = frame * dt
    
    # カラーコントアプロット（固定されたデータを使用）
    contour = ax_contour.contourf(X_fixed, T_fixed, Z_fixed, 
                                  levels=np.linspace(-1, 1, 21),
                                  cmap=cm.coolwarm, extend='both')
    
    # 等高線を追加
    ax_contour.contour(X_fixed, T_fixed, Z_fixed, 
                       levels=np.linspace(-1, 1, 11),
                       colors='black', alpha=0.3, linewidths=0.5)
    
    # 現在時刻を示す線（これだけが動く）
    ax_contour.axhline(y=current_time, color='green', linewidth=3, linestyle='-', label='Current time')
    
    # 監視点の位置を示す線
    ax_contour.axvline(x=x1, color='red', linewidth=1, linestyle=':', alpha=0.7)
    ax_contour.axvline(x=x2, color='blue', linewidth=1, linestyle=':', alpha=0.7)
    
    # 監視点での軌跡を強調
    ax_contour.plot(np.full_like(t_fixed, x1), t_fixed, 'w-', linewidth=2, alpha=0.5)
    ax_contour.plot(np.full_like(t_fixed, x2), t_fixed, 'w-', linewidth=2, alpha=0.5)
    
    ax_contour.set_xlabel('Position x [m]', fontsize=12)
    ax_contour.set_ylabel('Time t [s]', fontsize=12)
    ax_contour.set_title(f'Space-Time Wave Field (Contour Plot)', fontsize=14)
    ax_contour.set_xlim(-5, 10)
    ax_contour.set_ylim(t_start, t_end)
    
    # カラーバーを追加（初回のみ）
    if frame == 0:
        cbar = plt.colorbar(contour, ax=ax_contour)
        cbar.set_label('Amplitude', fontsize=12)
    
    # 右側のパネル：現在時刻での空間分布
    current_wave = A * np.sin(k*x - omega*current_time + phi)
    ax_time.plot(x, current_wave, 'k-', linewidth=2)
    
    # 監視点をマーク
    ax_time.plot(x1, A * np.sin(k*x1 - omega*current_time + phi), 'ro', markersize=10, label=f'x = {x1}')
    ax_time.plot(x2, A * np.sin(k*x2 - omega*current_time + phi), 'bo', markersize=10, label=f'x = {x2}')
    
    ax_time.set_xlabel('Position x [m]', fontsize=12)
    ax_time.set_ylabel('Amplitude', fontsize=12)
    ax_time.set_title(f'Spatial Distribution at t = {current_time:.2f} s', fontsize=14)
    ax_time.set_xlim(-5, 10)
    ax_time.set_ylim(-1.5, 1.5)
    ax_time.grid(True, alpha=0.3)
    ax_time.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_time.axvline(x=x1, color='red', linestyle=':', alpha=0.5)
    ax_time.axvline(x=x2, color='blue', linestyle=':', alpha=0.5)
    ax_time.legend()
    
    # 波の情報を表示
    phase_at_x1 = (k*x1 - omega*current_time + phi) % (2*np.pi)
    phase_at_x2 = (k*x2 - omega*current_time + phi) % (2*np.pi)
    info_text = f'Phase at x={x1}: {phase_at_x1:.2f} rad\nPhase at x={x2}: {phase_at_x2:.2f} rad'
    ax_time.text(0.05, 0.95, info_text, transform=ax_time.transAxes, 
                verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    return []

# アニメーションの作成
anim = animation.FuncAnimation(fig_anim, animate, init_func=init_anim,
                              frames=n_frames, interval=50,
                              blit=False, repeat=True)

# HTMLとして表示
HTML(anim.to_jshtml())

# 2: F-K変換

F-K変換は、波の振幅を空間と時間の関数として表現する方法です。波の振幅 $a(x, t)$ をフーリエ変換すると、以下のように表されます。

$$
\begin{align*}
A(\omega_i, x) &= \frac{1}{N} \sum_{n=0}^{N-1} a(x, t_n) e^{-i \omega_i t_n} \hspace{30pt} (2-3) \\
A(k_i, \omega_i) &= \frac{1}{M} \sum_{m=0}^{M-1} A(\omega_i, x_m) e^{-i k_i x_m} \hspace{30pt} (2-4)
\end{align*}
$$

以下のコードでは、F-K変換を用いて波の振幅を空間と時間の関数として表現し、アニメーションで表示します。

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
from IPython.display import HTML
import matplotlib

# F-K変換の可視化
# (x,t)空間から(k,ω)空間への変換を理解する

# パラメータ設定
# 複数の波を重ね合わせる（異なる速度の波）
wave_params = [
    {'A': 1.0, 'k': 1.0, 'omega': 2.0, 'phi': 0.0, 'c': 2.0},      # 波1: c = 2.0 m/s
    {'A': 0.7, 'k': 1.5, 'omega': 1.5, 'phi': np.pi/4, 'c': 1.0},  # 波2: c = 1.0 m/s  
    {'A': 0.5, 'k': 0.5, 'omega': 1.5, 'phi': -np.pi/3, 'c': 3.0}  # 波3: c = 3.0 m/s
]

# 空間と時間のグリッド
nx, nt = 128, 128
x_max, t_max = 20, 10
x = np.linspace(0, x_max, nx)
t = np.linspace(0, t_max, nt)
dx = x[1] - x[0]
dt = t[1] - t[0]

# 波動場の生成（複数の波の重ね合わせ）
X, T = np.meshgrid(x, t)
wave_field = np.zeros_like(X)

print("波のパラメータ:")
for i, params in enumerate(wave_params):
    wave_i = params['A'] * np.sin(params['k']*X - params['omega']*T + params['phi'])
    wave_field += wave_i
    print(f"  波{i+1}: A={params['A']:.1f}, k={params['k']:.1f} rad/m, ω={params['omega']:.1f} rad/s, c={params['c']:.1f} m/s")

# F-K変換の実行
# Step 1: 時間方向のFFT（各位置で）
fft_t = np.fft.fft(wave_field, axis=0)
omega_array = 2 * np.pi * np.fft.fftfreq(nt, dt)

# Step 2: 空間方向のFFT（各周波数で）
fft_xt = np.fft.fft(fft_t, axis=1)
k_array = 2 * np.pi * np.fft.fftfreq(nx, dx)

# F-Kスペクトルの計算（パワー）
fk_spectrum = np.abs(fft_xt)**2

# 正の周波数・波数領域のみ抽出
omega_positive = omega_array[:nt//2]
k_positive = k_array[:nx//2]
fk_spectrum_positive = fk_spectrum[:nt//2, :nx//2]

# 静的な可視化
fig = plt.figure(figsize=(16, 10))

# 1. 時空間波動場
ax1 = fig.add_subplot(2, 3, 1)
im1 = ax1.imshow(wave_field, extent=[0, x_max, t_max, 0], 
                 cmap='coolwarm', aspect='auto', vmin=-2, vmax=2)
ax1.set_xlabel('Position x [m]')
ax1.set_ylabel('Time t [s]')
ax1.set_title('Wave Field in (x,t) Space')
plt.colorbar(im1, ax=ax1, label='Amplitude')

# 理論的な波の軌跡を追加
for params in wave_params:
    # 波の軌跡: x = ct + x0
    t_line = np.linspace(0, t_max, 100)
    x_line = params['c'] * t_line
    mask = x_line <= x_max
    ax1.plot(x_line[mask], t_line[mask], '--', linewidth=1, alpha=0.7)

# 2. F-Kスペクトル（2D）
ax2 = fig.add_subplot(2, 3, 2)
im2 = ax2.imshow(np.log10(fk_spectrum_positive + 1e-10), 
                 extent=[0, k_positive.max(), 0, omega_positive.max()],
                 cmap='viridis', aspect='auto', origin='lower')
ax2.set_xlabel('Wave number k [rad/m]')
ax2.set_ylabel('Angular frequency ω [rad/s]')
ax2.set_title('F-K Spectrum (log scale)')
plt.colorbar(im2, ax=ax2, label='log10(Power)')

# 理論的な分散関係を追加
k_theory = np.linspace(0, 3, 100)
for params in wave_params:
    omega_theory = params['c'] * k_theory
    ax2.plot(k_theory, omega_theory, 'w--', linewidth=1, alpha=0.7)

# 3. 特定周波数での波数スペクトル
ax3 = fig.add_subplot(2, 3, 3)
omega_idx = np.argmin(np.abs(omega_positive - 1.5))  # ω ≈ 1.5 rad/s
k_spectrum_at_omega = fk_spectrum_positive[omega_idx, :]
ax3.plot(k_positive, k_spectrum_at_omega)
ax3.set_xlabel('Wave number k [rad/m]')
ax3.set_ylabel('Power')
ax3.set_title(f'Wave Number Spectrum at ω = {omega_positive[omega_idx]:.2f} rad/s')
ax3.grid(True, alpha=0.3)

# 4. 時間領域の信号（特定位置）
ax4 = fig.add_subplot(2, 3, 4)
x_idx = nx//4  # x = 5 m付近
time_signal = wave_field[:, x_idx]
ax4.plot(t, time_signal)
ax4.set_xlabel('Time t [s]')
ax4.set_ylabel('Amplitude')
ax4.set_title(f'Time Signal at x = {x[x_idx]:.1f} m')
ax4.grid(True, alpha=0.3)

# 5. 周波数スペクトル（特定位置）
ax5 = fig.add_subplot(2, 3, 5)
freq_spectrum = np.abs(fft_t[:nt//2, x_idx])
ax5.plot(omega_positive, freq_spectrum)
ax5.set_xlabel('Angular frequency ω [rad/s]')
ax5.set_ylabel('Amplitude')
ax5.set_title(f'Frequency Spectrum at x = {x[x_idx]:.1f} m')
ax5.grid(True, alpha=0.3)

# 6. 位相速度の抽出
ax6 = fig.add_subplot(2, 3, 6)
# F-Kスペクトルのピークから位相速度を推定
for i in range(len(omega_positive)):
    if omega_positive[i] > 0.5:  # 低周波ノイズを除外
        k_peak_idx = np.argmax(fk_spectrum_positive[i, :])
        if k_positive[k_peak_idx] > 0:
            c_estimated = omega_positive[i] / k_positive[k_peak_idx]
            ax6.scatter(omega_positive[i], c_estimated, c='blue', alpha=0.5, s=10)

ax6.set_xlabel('Angular frequency ω [rad/s]')
ax6.set_ylabel('Phase velocity c [m/s]')
ax6.set_title('Extracted Phase Velocities')
ax6.set_ylim(0, 5)
ax6.grid(True, alpha=0.3)

# 理論値を追加
for i, params in enumerate(wave_params):
    ax6.axhline(y=params['c'], color=f'C{i}', linestyle='--', 
                label=f"Wave {i+1}: c = {params['c']} m/s")
ax6.legend()

plt.tight_layout()

# アニメーション版：F-K変換のプロセスを可視化
fig_anim = plt.figure(figsize=(16, 8))

ax_space = fig_anim.add_subplot(1, 3, 1)
ax_freq = fig_anim.add_subplot(1, 3, 2)
ax_fk = fig_anim.add_subplot(1, 3, 3)

# アニメーション用の関数
def init():
    ax_space.clear()
    ax_freq.clear()
    ax_fk.clear()
    return []

def animate(frame):
    ax_space.clear()
    ax_freq.clear()
    ax_fk.clear()
    
    # 現在の時刻
    current_time = frame * 0.1
    t_idx = int(current_time / dt) % nt
    
    # 1. 現在時刻での空間分布
    spatial_dist = wave_field[t_idx, :]
    ax_space.plot(x, spatial_dist, 'b-', linewidth=2)
    ax_space.set_xlabel('Position x [m]')
    ax_space.set_ylabel('Amplitude')
    ax_space.set_title(f'Spatial Distribution at t = {current_time:.1f} s')
    ax_space.set_ylim(-3, 3)
    ax_space.grid(True, alpha=0.3)
    
    # 2. 空間分布のフーリエ変換
    fft_spatial = np.fft.fft(spatial_dist)
    k_spectrum = np.abs(fft_spatial[:nx//2])
    ax_freq.plot(k_positive, k_spectrum, 'r-', linewidth=2)
    ax_freq.set_xlabel('Wave number k [rad/m]')
    ax_freq.set_ylabel('Amplitude')
    ax_freq.set_title('Spatial Fourier Transform')
    ax_freq.set_ylim(0, nx/2)
    ax_freq.grid(True, alpha=0.3)
    
    # 3. F-Kスペクトル（現在時刻までのデータ）
    if t_idx > 10:
        wave_field_current = wave_field[:t_idx, :]
        fft_t_current = np.fft.fft(wave_field_current, axis=0)
        fft_xt_current = np.fft.fft(fft_t_current, axis=1)
        fk_spectrum_current = np.abs(fft_xt_current)**2
        
        omega_current = 2 * np.pi * np.fft.fftfreq(t_idx, dt)
        omega_pos_current = omega_current[:t_idx//2]
        fk_pos_current = fk_spectrum_current[:t_idx//2, :nx//2]
        
        im = ax_fk.imshow(np.log10(fk_pos_current + 1e-10),
                         extent=[0, k_positive.max(), 0, omega_pos_current.max()],
                         cmap='viridis', aspect='auto', origin='lower')
        ax_fk.set_xlabel('Wave number k [rad/m]')
        ax_fk.set_ylabel('Angular frequency ω [rad/s]')
        ax_fk.set_title(f'F-K Spectrum (0 to {current_time:.1f} s)')
        
        # 理論的な分散関係
        for params in wave_params:
            k_theory = np.linspace(0, 3, 50)
            omega_theory = params['c'] * k_theory
            ax_fk.plot(k_theory, omega_theory, 'w--', linewidth=1, alpha=0.5)
    
    return []

# アニメーションの作成
anim = animation.FuncAnimation(fig_anim, animate, init_func=init,
                              frames=100, interval=100,
                              blit=False, repeat=True)

# HTMLとして表示
HTML(anim.to_jshtml())